In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [ ]:
!pip install -q supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive # подключаем гугл драйв диск для загрузки файлов
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
from tqdm.notebook import tqdm
import supervision as sv
import numpy as np
from ultralytics import YOLO

VIDEO_PATH = '/content/drive/MyDrive/Стажировка/video_test/rakurs2.mp4'

RESULT_PATH = '/content/drive/MyDrive/Стажировка/video_test/RESULT/result2.mp4'

model = YOLO("/content/drive/MyDrive/Стажировка/weights/best_120e.pt")

# Информация о видео
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)

In [ ]:
print(video_info)

VideoInfo(width=2048, height=1536, fps=100, total_frames=1127)


In [ ]:
# Цвет Аннотатора
#color = sv.Color.from_hex("#f00")
color_palette = sv.ColorPalette.from_hex(["#0f0", "#00f", "#f00"])
# Цвет текста Аннотатора
text_color = sv.Color.from_hex("#0ff")
# Создаем трэкер
byte_tracker = sv.ByteTrack()
# Создаем Аннотатор
#label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_CENTER, color=color_palette, text_color=text_color, text_padding=2, text_thickness=1)
#corner_annotator = sv.BoxCornerAnnotator(color=color_palette, thickness=1)
#ellipse_annotator = sv.EllipseAnnotator(color=color, thickness=2)
#box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=2, text_scale=1)
triangle_annotator = sv.TriangleAnnotator(color = color_palette)

In [ ]:
# Продолжительность сек.
print(f'Res: {video_info.resolution_wh}, FPS: {video_info.fps}, Dur.: {video_info.total_frames//video_info.fps} sec.')

Res: (2048, 1536), FPS: 100, Dur.: 11 sec.


### Способ 1

In [ ]:
'''
Функция колбэка для обработки кадра
'''
def process_frame(frame: np.ndarray, _) -> np.ndarray:

    # Предсказание модели
    results = model(frame, imgsz=video_info.width, conf=0.4, verbose=False)[0]

    # Разбираем результат
    detections = sv.Detections.from_ultralytics(results)

    # Обновляем трэкер
    detections = byte_tracker.update_with_detections(detections)

    # Фильтр по номеру класса 2 - без каски
    detections = detections[np.isin(detections.class_id, [2,5])]

    # Формируем метку для Аннотатора
    #labels = [f"#{tracker_id} {confidence:.0%}" for _, _, confidence, _, tracker_id in detections]
    # labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _ in detections]

    # Аннотируем фрейм
    #frame = label_annotator.annotate(scene=frame.copy(), detections=detections, labels=labels)
    frame = triangle_annotator.annotate(scene=frame, detections=detections)
    frame = ellipse_annotator.annotate(scene=frame, detections=detections)

    return frame

In [ ]:
# Обработка видео
sv.process_video(source_path=VIDEO_PATH, target_path=RESULT_PATH, callback=process_frame)

### Способ 2

In [ ]:
# Создаем генератор фреймов
frames_generator = sv.get_video_frames_generator(VIDEO_PATH)
# Создаем итератор по генератору
frame_iterator = iter(frames_generator)

with sv.VideoSink(target_path=RESULT_PATH, video_info=video_info) as sink:
  # Прикрутим tqdm в цикл для вывода прогресса
  for frame in tqdm(frame_iterator, total=video_info.total_frames):

      # Предиктим
      result = model(frame, imgsz=video_info.width, verbose=False)[0]
      # Детектим
      detections = sv.Detections.from_ultralytics(result)

      #detections = byte_tracker.update_with_detections(detections)

      nh_detections = detections[np.isin(detections.class_id, [0,2])]
      #h_detections = detections[np.isin(detections.class_id, [2])]

      #labels = [f"id{tracker_id} {confidence:.0%}" for _, _, confidence, _, tracker_id in h_detections]

      #annotated_frame = label_annotator.annotate(scene=frame.copy(), detections=h_detections, labels=labels)

      annotated_frame = triangle_annotator.annotate(scene=frame.copy(), detections=nh_detections)
      #annotated_frame = ellipse_annotator.annotate(scene=annotated_frame.copy(), detections=p_detections)
      #annotated_frame = corner_annotator.annotate(scene=annotated_frame.copy(), detections=h_detections)

      sink.write_frame(frame=annotated_frame)

  0%|          | 0/5205 [00:00<?, ?it/s]